In [1]:
import time
import torch

import pathpy as pp2
import pathpyG as pp
from matplotlib import pyplot as plt


print('Running on', pp.config['torch']['device'])

Running on cpu


In [17]:
x = torch.nested.as_nested_tensor([
    torch.tensor([
        [0,1,2], 
        [1,2,3]
    ]), 
    torch.tensor([
        [0,1], 
        [1,2]
    ]),
    torch.tensor([
        [2,3,4,5],
        [3,4,5,6],
    ]),
    ], device=pp.config['torch']['device'])
f = torch.tensor([1, 2, 4])

In [18]:
torch.cat([t for t in x], dim=1)

tensor([[0, 1, 2, 0, 1, 2, 3, 4, 5],
        [1, 2, 3, 1, 2, 3, 4, 5, 6]])

In [32]:
torch.cat([f[i].repeat(t.size(dim=1)) for i, t in enumerate(x)])

tensor([1, 1, 1, 2, 2, 2, 3, 3, 3, 3])

In [29]:
f[0].repeat(5)

tensor([1, 1, 1, 1, 1])

In [8]:
for t in x:
    print(t)

tensor([[0, 1, 2],
        [1, 2, 3]])
tensor([[0, 1],
        [1, 2]])
tensor([[2, 3, 4, 5],
        [3, 4, 5, 6]])


In [2]:
def nested_unfold(nt, k):
    a = []
    for t in nt:
        if k < t.size(dim=1):
            a.append(t.unfold(1, k, 1))
        else:
            a.append(torch.tensor([], dtype=int, device=pp.config['torch']['device']))    
    return torch.cat(a, dim=1)

In [3]:
i = nested_unfold(x, 2)
print(i)

NameError: name 'x' is not defined

In [28]:
f_q = torch.cat([f[i].repeat(t.size(dim=1)-2+1 if t.size(dim=1)-2+1>0 else 0) for i, t in enumerate(x)])
print(f_q)

tensor([1, 1, 2, 4, 4, 4])


In [29]:
edge_index, reverse_index = i.unique(dim=1, return_inverse=True)
print(edge_index)
print(reverse_index)

tensor([[[0, 1],
         [1, 2],
         [2, 3],
         [3, 4],
         [4, 5]],

        [[1, 2],
         [2, 3],
         [3, 4],
         [4, 5],
         [5, 6]]])
tensor([0, 1, 2, 3, 4])


In [31]:
torch.bincount(reverse_index, weights=f_q[reverse_index])

tensor([1., 1., 2., 4., 4.], dtype=torch.float64)

In [2]:
pp.config['torch']['device'] = 'cpu'
p = pp.PathData.from_csv('../data/tube_paths_train.ngram')

/opt/conda/lib/python3.10/site-packages/torch/nested/__init__.py:58: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at ../aten/src/ATen/NestedTensorImpl.cpp:178.)
  return torch._nested_tensor_from_tensor_list(tensor_list, dtype, None, device, None)


In [6]:
g2 = pp.HigherOrderGraph(p, order=2, path_freq='path_freq')
print(g2.N)
print(g2.M)
print(g2['edge_weight'].sum().item())

In [13]:
print(p.path_freq.size())

torch.Size([61748])


In [6]:
pp.config['torch']['device'] = 'cpu'
p = pp.PathData.from_csv('../data/tube_paths_train.ngram')

In [7]:
g2 = pp.HigherOrderGraph(p, order=7, path_freq='path_freq')
print(g2.N)
print(g2.M)
print(g2['edge_weight'].sum().item())

4056
5332
340694


In [ ]:
pp.config['torch']['device'] = 'cuda'
p = pp.PathData.from_csv('../data/tube_paths_train.ngram')

In [ ]:
g2 = pp.HigherOrderGraph(p, order=4, path_freq='path_freq')
print(g2.N)
print(g2.M)
print(g2['edge_weight'].sum().item())

In [ ]:
for e in g2.edges:
    print(e, g2['edge_weight', e[0], e[1]])

In [ ]:
g2['edge_weight', ('Southwark', 'Waterloo'), ('Waterloo', 'Embankment')]

In [ ]:
paths = pp2.Paths.read_file("../data/tube_paths_train.ngram", max_subpath_length=2)
g2 = pp2.HigherOrderNetwork(paths, k=2)
print(g2)

# Pathpy 2.0

In [ ]:
ks = range(1,10)
times = []
for k in ks:
    start = time.time() 
    paths = pp2.Paths.read_file("../data/tube_paths_train.ngram", max_subpath_length=k)
    g2 = pp2.HigherOrderNetwork(paths, k=k)
    print(g2)
    elapsed_pp = time.time()-start
    times.append(elapsed_pp)
plt.plot(ks, times)

# pathpyG (GPU)

In [ ]:
pp.config['torch']['device'] = 'cuda:0'

In [ ]:
ks = range(1,10)
times_new_gpu = []
for k in ks:
    start = time.time()
    p = pp.PathData.from_csv('../data/tube_paths_train.ngram')
    gk = pp.HigherOrderGraph(p, order=k, path_freq='path_freq')
    print(gk)
    print('---')
    elapsed_new = time.time()-start
    times_new_gpu.append(elapsed_new)

In [ ]:
pp.config['torch']['device'] = 'cpu'

In [ ]:
ks = range(1,10)
times_new_cpu = []
for k in ks:
    start = time.time()
    p = pp.PathData.from_csv('../data/tube_paths_train.ngram')
    gk = pp.HigherOrderGraph(p, order=k, path_freq='path_freq')
    print(gk)
    print('---')
    elapsed_new = time.time()-start
    times_new_cpu.append(elapsed_new)

In [ ]:
plt.plot(ks, times, label='pathpy2')
plt.plot(ks, times_new_gpu, label='pathpyG prototype (GPU)')
plt.plot(ks, times_new_cpu, label='pathpyG prototype (CPU)')
plt.xlabel('order')
plt.grid()
plt.ylabel('time [s]')
plt.legend()

In [ ]:
plt.plot(ks, times, label='pathpy2')
plt.plot(ks, times_new_gpu, label='pathpyG prototype (GPU)')
plt.plot(ks, times_new_cpu, label='pathpyG prototype (CPU)')
plt.xlabel('order')
plt.ylabel('time [s]')
plt.legend()
plt.grid()
plt.yscale('log')
plt.ylim(1,100)